<a href="https://www.kaggle.com/code/mohamedalaaelsayad/amzon-food-text-cleaning?scriptVersionId=203944332" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")
df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
df.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [4]:
df.shape

(568454, 10)

In [5]:
df["Text"].isnull().sum()

0

# **start with Lowercase all text**

In [6]:
df["Text"] = df["Text"].str.lower()
df["Text"].sample(5)

225928    these cashews are in small pieces - some very ...
460192    this was the best price i found for the gu pac...
547046    i read quite a bit of reviews before buying th...
529780    this bold robust coffee is my new favorite!  r...
361904    i don't like bitter coffee, which i associate ...
Name: Text, dtype: object

# **now, let's Correcr the Spelling mistakes**

In [7]:
! pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.1 MB/s eta 0:00:00


In [8]:
from spellchecker import SpellChecker 
import re

def correct_spelling(text) : 
    text = re.sub(r"[^a-z\s]" , " " , text)
    spell = SpellChecker()
    corrected_text = []
    for word in text.split() : 
        correct = spell.correction(word)
        if correct is None : 
            correct = word
        corrected_text.append(correct)
    clean_text = " ".join(corrected_text)
    return clean_text

In [9]:
#df["Text"].apply(correct_spelling)
#df["Text"].sample(5)

# **remove html tags**

In [10]:
from bs4 import BeautifulSoup

def remove_html_tags(text) : 
    soup = BeautifulSoup(text , "html.parser")
    claen_text = soup.get_text()
    return claen_text

In [11]:
df["Text"].apply(remove_html_tags)
df["Text"].sample(5)

/tmp/ipykernel_23/1427653923.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text , "html.parser")


185913    this powder makes a wondeful chili.  not too h...
163679    i was hoping for a ecological solution to the ...
475432    these are great! i send the kiddos off to pres...
480842    better then expected regardless misleading cou...
456208    i like to try new things and when i came acros...
Name: Text, dtype: object

# **remove urls**

In [12]:
def romove_url(text) : 
    url_pattern = re.compile(r"https?://\s+|www.\s")
    clean_text = re.sub(url_pattern , " " , text)
    return clean_text

In [13]:
df["Text"].apply(romove_url)

0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price.  there was a wid...
                                ...                        
568449    great for sesame chicken..this is a good if no...
568450    i'm disappointed with the flavor. the chocolat...
568451    these stars are small, so you can give 10-15 o...
568452    these are the best treats for training and rew...
568453    i am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object

# **now , we have to convert all numbers to text**

In [14]:
!pip install num2words

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 3.6 MB/s eta 0:00:00


In [15]:
from num2words import num2words
def convert_num_to_word(text) : 
    numbers = re.findall(r"\d+" , text)
    for number in numbers :
        text = text.replace(number, num2words(int(number)))
    return text
        

In [16]:
df["Text"].apply(convert_num_to_word)

0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price.  there was a wid...
                                ...                        
568449    great for sesame chicken..this is a good if no...
568450    i'm disappointed with the flavor. the chocolat...
568451    these stars are small, so you can give ten-fif...
568452    these are the best treats for training and rew...
568453    i am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object

# **convert chat words to normal words**

In [17]:
chat_words = {
    "AFAIK": "As Far As I Know",
    "OMG" : "oh my God",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laughter",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "LOL": "Laughing out loud",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don’t care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "IDC": "I don’t care",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "LMAO": "Laughing my a** off",
    "BFF": "Best friends forever",
    "CSL": "Can’t stop laughing",
}

In [18]:
def convert_chat_words(text) : 
    words = text.split()
    converted_words = [chat_words.get(word, word) for word in words]
    clean_text = " ".join(converted_words)
    return clean_text

In [19]:
text = "OMG , it's abdAlbaset Hamooda!"
correct_text = convert_chat_words(text)
print(correct_text)

oh my God , it's abdAlbaset Hamooda!


In [20]:
df["Text"].apply(convert_chat_words)

0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price. there was a wide...
                                ...                        
568449    great for sesame chicken..this is a good if no...
568450    i'm disappointed with the flavor. the chocolat...
568451    these stars are small, so you can give 10-15 o...
568452    these are the best treats for training and rew...
568453    i am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object

In [21]:
contractions_dict = {
        "can't": "cannot",
        "won't": "will not",
        "n't": " not",
        "'re": " are",
        "'s": " is",
        "'d": " would",
        "'ll": " will",
        "'t": " not",
        "'ve": " have",
        "'m": " am",
        "i'm": "I am",
        "it's": "it is",
        "he's": "he is",
        "she's": "she is",
        "we're": "we are",
        "they're": "they are",
        "there's": "there is",
        "who's": "who is",
        "that's": "that is",
        "what's": "what is",
        "let's": "let us",
        "i've": "I have",
        "we've": "we have",
        "they've": "they have",
        "should've": "should have",
        "could've": "could have",
        "would've": "would have",
        "you've": "you have",
        "he'd": "he would",
        "she'd": "she would",
        "we'd": "we would",
        "they'd": "they would",
        "i'd": "I would"
    }

In [22]:
def convert_chat_words(text) : 
    words = text.split()
    converted_words = [contractions_dict.get(word, word) for word in words]
    clean_text = " ".join(converted_words)
    return clean_text

In [23]:
text = "OMG , it's abdAlbaset Hamooda!"
correct_text = convert_chat_words(text)
print(correct_text)

OMG , it is abdAlbaset Hamooda!


In [24]:
df["Text"].apply(convert_chat_words)

0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price. there was a wide...
                                ...                        
568449    great for sesame chicken..this is a good if no...
568450    I am disappointed with the flavor. the chocola...
568451    these stars are small, so you can give 10-15 o...
568452    these are the best treats for training and rew...
568453    i am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object